In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
from tqdm import tqdm
from pathlib import Path
from src.utils.config_loader import load_config

base_dir = Path(os.getcwd()).parent

config = load_config(base_dir / 'secrets.yaml')

In [3]:
from src.data.preprocessing import create_df

val_df = create_df(base_dir / 'data/my_data/regplans-dev.conllu')

In [4]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import (SystemMessage, HumanMessage)

os.environ['OPENAI_API_VERSION'] = config['OPENAI_API_VERSION']
os.environ['AZURE_OPENAI_ENDPOINT'] = config['OPENAI_API_BASE']
os.environ['AZURE_OPENAI_API_KEY'] = config['OPENAI_API_KEY']

llm = AzureChatOpenAI(
    deployment_name=config['OPENAI_DEPLOYMENT_NAME'],
    temperature=0.0
)

In [5]:
import json

def format_examples(example_subset): 
    # Formats the examples into a string for later prompt
    formatted = []
    for i, ex in enumerate(example_subset):
        entity_lines = "\n".join([f"{e['word']} {e['label']}" for e in ex["entities"]])
        formatted.append(f"Example {i+1}:\nSentence: \"{ex['sentence']}\"\nEntities:\n{entity_lines}\n")
    
    return "\n".join(formatted)

with open(base_dir / 'llm_stuff/prompts/examples.json', 'r') as f:
    example_bank = json.load(f)

ids = [24, 20] 

examples = [next(ex for ex in example_bank if ex["id"] == id) for id in ids]

formatted_examples = format_examples(examples)

print(formatted_examples)

Example 1:
Sentence: "Områdene er felles for byggeområdene BFS1 , BFS2 , BFS3 , BFS4 , BFS5 , B1 , B2 og BKS1 og skal nyttes til lek og opphold / rekreasjon ."
Entities:
BFS1 B-FELT
BFS2 B-FELT
BFS3 B-FELT
BFS4 B-FELT
BFS5 B-FELT
B1 B-FELT
B2 B-FELT
BKS1 B-FELT

Example 2:
Sentence: "Det avsettes 2 biloppstillingsplasser pr . boenhet i BKS1-BKS6 og BFS1-BFS3 ."
Entities:
BKS1-BKS6 B-FELT
BFS1-BFS3 B-FELT



In [6]:
from src.utils.label_mapping_regplans import label_to_id
from collections import defaultdict

all_pred_ids = []
all_true_ids = []
all_results = []

# Only do the anlaysis on 25 % of the data
# val_df = val_df.iloc[:int(len(val_df) * 0.25)]

for idx, row in tqdm(val_df.iterrows(), total=len(val_df)):

    sentence = row['full_text']
    tokens = row['words']
    true_labels = row['labels']  

    msg = [
    SystemMessage(
        f"""You are an expert in Natural Language Processing. Your task is to identify Named Entities (NER) in a given text.
            The possible Named Entities are exclusively 'B-FELT' and 'I-FELT'. The entities are defined as follows:

            - B-FELT: The beginning of a field zone name.
            - I-FELT: The continuation of a field zone name.   
                        
            ### Examples:
            
            {formatted_examples}
        """
    ),
    HumanMessage(f"Your task is to identify the Named Entities in the following sentence: '{sentence}'") ]

    response = llm.invoke(msg)

    entities = defaultdict(list) # Word-label pairs

    for line in response.content.splitlines():
        parts = line.strip().split()
        if len(parts) == 2:
            word, label = parts[0], parts[1]
            entities[word].append(label)

    pred_labels = []
    word_counts = defaultdict(int)  # Track occurrences of each word

    for token in tokens:
        if token in entities and word_counts[token] < len(entities[token]):
            pred_labels.append(entities[token][word_counts[token]])  # Get the label in order
            word_counts[token] += 1  # Increment occurrence counter
        else:
            pred_labels.append("O")  # Default to "O" if missing

    # Convert labels to IDs
    pred_ids = []
    for label in pred_labels:
        if label in label_to_id:
            pred_ids.append(label_to_id[label])
        else:
            pred_ids.append(label_to_id.get("O", -1))

    true_ids = [label_to_id[label] for label in true_labels]

    all_pred_ids.extend(pred_ids)
    all_true_ids.extend(true_ids)

    all_results.append({
        'sentence': sentence,
        'tokens': tokens,
        'true_labels': true_labels,
        'predicted_labels': pred_labels,
        'generated_text': response.content
    })

  0%|          | 0/353 [00:00<?, ?it/s]

100%|██████████| 353/353 [32:17<00:00,  5.49s/it]  


In [7]:
from llm_stuff.evaluation import evaluate 

metrics = evaluate(all_true_ids, all_pred_ids)

print("Evaluation Metrics on Dev Set:")
print(metrics)

final_output = {
    'prompt': str(msg),
    'evaluation_metrics': metrics,
    'results': all_results
}

with open(base_dir / f"llm_stuff/results/{config['OPENAI_DEPLOYMENT_NAME']}_fewShot_2ex.json", 'w', encoding='utf-8') as f:
    json.dump(final_output, f, indent=4, ensure_ascii=False)

Evaluation Metrics on Dev Set:
{'precision': 0.6279045939445496, 'recall': 0.616289496421814, 'f1': 0.6190614104270935, 'span_acc': 0.7322834730148315, 'classification_report': {'B-FELT': {'precision': 0.8333333333333334, 'recall': 0.7480314960629921, 'f1-score': 0.7883817427385892, 'support': 127.0}, 'I-FELT': {'precision': 0.0625, 'recall': 0.1111111111111111, 'f1-score': 0.08, 'support': 9.0}, 'O': {'precision': 0.987880671224363, 'recall': 0.9897260273972602, 'f1-score': 0.9888024883359253, 'support': 3212.0}, 'accuracy': 0.9781959378733572, 'macro avg': {'precision': 0.6279046681858987, 'recall': 0.6162895448571212, 'f1-score': 0.6190614103581714, 'support': 3348.0}, 'weighted avg': {'precision': 0.9795306300197095, 'recall': 0.9781959378733572, 'f1-score': 0.9787568918347649, 'support': 3348.0}}}
